# Mask RCNN Model

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Root directory of the project
ROOT_DIR = os.path.abspath("../Mask_RCNN_TF2")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.model import log

#sys.path.append('./samples/cork')
import corks

#sys.path.append(ROOT_DIR)

%matplotlib inline 

import datetime
import time
import numpy as np
import skimage.draw
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from corks import CorkDataset
import matplotlib.pyplot as plt

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

class InferenceConfig(corks.CorkConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
config = InferenceConfig()

config.DETECTION_MIN_CONFIDENCE = 0.7

model_mrcnn = modellib.MaskRCNN(mode="inference", config=config, model_dir='./logs')
# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = '../Mask_RCNN_TF2/logs/cork20210429T1106_all_aug_withLR0001/mask_rcnn_cork_0060.h5'
#model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model_mrcnn.load_weights(model_path, by_name=True)

dataset_val = CorkDataset()
dataset_val.load_cork('../Mask_RCNN_TF2/datasets/cork', "val")
dataset_val.prepare()

print("Running evaluation on test images.")

mrgData={}
t_prediction = 0
t_start = time.time()

for i, image_id in enumerate(dataset_val.image_ids):
        # Load image and run detection
        image = dataset_val.load_image(image_id)
        original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, config, image_id)
        #log("original_image", original_image)
        #log("image_meta", image_meta)
        #log("gt_class_id", gt_class_id)
        #log("gt_bbox", gt_bbox)
        #log("gt_mask", gt_mask)
        print("ORINGINAL IMG SHAPE", original_image.shape)
        
        fPath = dataset_val.image_info[image_id]['path']
        words = fPath.split('/')
        fname = words[-1]
        fnameGT='GT'+str(i)+fname
        fnameP='P'+str(i)+fname
        visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, dataset_val.class_names, title=fnameGT, figsize=(8, 8))
        
        
        #plt.savefig(os.path.join('GroundT',fname),bbox_inches='tight')
        print("Image File Name", i)

        # Detect objects
        t = time.time()
        
        r = model_mrcnn.detect([image], verbose=0)[0]
        #print("PRED SHAPE", r['masks'].shape)
        t_prediction += (time.time() - t)
        print("Pred Time:", (time.time() - t))
        fname_json=fname.split('.')[0] + '.json'
        print(fname_json)
        visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], title=fnameP, ax=get_ax())
        #plt.savefig(os.path.join('Prediction',fname),bbox_inches='tight')
        
        r['masks']=r['masks'].astype(int).tolist()
        r['rois']=r['rois'].tolist()
        r['class_ids']=r['class_ids'].tolist()
        r['scores']=r['scores'].tolist()
        
        
        #with open(os.path.join('json_predictions',fname_json), 'w') as outfile:
        #    json.dump(r, outfile)
        

print("Prediction time: {}. Average {}/image".format(t_prediction, t_prediction / len(dataset_val.image_ids)))
print("Total time: ", time.time() - t_start)

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
#image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
A_F1 = []
A_Prec = []
A_Rec = []
iou_scores=[]
totalTP = 0
totalFP = 0
totalFN = 0
for i,image_id in enumerate(dataset_val.image_ids):
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, config,
                               image_id)
    #molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
    # Run object detection
    t = time.time()
    results = model_mrcnn.detect([image], verbose=0)
    pred_time = time.time() - t
    r = results[0]
    
    # Compute AP
    gt_match,pred_match,overlap = utils.compute_matches(gt_bbox, gt_class_id, gt_mask,
        r["rois"], r["class_ids"], r["scores"], r['masks'])
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    
    prec, rec, f1 = utils.compute_f1_score(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    
    fPath = dataset_val.image_info[image_id]['path']
    words = fPath.split('/')
    fname = words[4]
    
    iou=utils.compute_overlaps_masks(gt_mask,r['masks'])
    if len(iou)==0:
        iou=0
    else:
        iou=np.sum(iou,axis=1)
        iou=np.mean(iou)
    print("IoU Score mean",iou)
    iou_scores.append(iou)
    FP = np.count_nonzero(pred_match == -1)
    TP = len(pred_match) - FP
    FN = np.count_nonzero(gt_match == -1)
    print("File Name: ", i)
    #print("GT",gt_match)
    #print("PRED",pred_match)
    if FP == 0 and TP == 0 and FN == 0:
      AP = 1.0
    elif len(gt_match)==0 and FP > 0:
      AP = 0.0
    elif len(pred_match)==0 and FN > 0:
      AP = 0.0
    #print("AP", AP)
    print("Prediction Time:",pred_time)
    print("False Positive:",FP)
    print("True Positive:",TP)
    print("False Negative:",FN)
    print("Precision",prec)
    print("Recall",rec)
    print("F1-score",f1)
    
    print("AP",AP)
    #try:
    #    p = TP / (TP + FP)
    #    r = TP / (TP + FN)
    #except:
    #    if FP == 0 and TP == 0 and FN == 0:
    #        p = 1
    #        r = 1
    #    else:
    #        p = 0
    #        r = 1
    #    pass
    #print("PRECISION MAN",p)
    #print("RECALL MAN",r)
    
    totalTP = totalTP + TP
    totalFP = totalFP + FP
    totalFN = totalFN + FN
    print("####################################################")
    APs.append(AP)
    A_F1.append(f1)
    A_Prec.append(prec)
    A_Rec.append(rec)
    
print("Summary:-")
print("Confidence Threshold:", config.DETECTION_MIN_CONFIDENCE)
print("Total True Positive:", totalTP)
print("Total False Positive:", totalFP)
print("Total False Negative:", totalFN)
print("Average Recall:", np.mean(A_Rec))
print("Average Precision:", np.mean(A_Prec))
print("Mean F1-Score:", np.mean(A_F1))
print("Average IoU Score",np.mean(iou_scores))
print("mAP: ", np.mean(APs))

# U-Net

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["SM_FRAMEWORK"] = "tf.keras"
ROOT_DIR = os.path.abspath("../segmentation_models")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
import json
import cv2
import skimage.draw
import skimage.color
import skimage.io
import skimage.transform
from skimage import morphology as morph
from skimage.io import imread, imsave
from skimage.color import rgb2gray
import keras
import numpy as np
import pickle
import matplotlib.pyplot as plt
import albumentations as A
import segmentation_models as sm

In [ ]:
#Using absolute paths
DATA_DIR = '/tf/notebooks/hdd_1/sen/MyCode/segmentation_models/data/cork'

# load repo with data if it is not exists
if not os.path.exists(DATA_DIR):
    print('Loading data...')
    os.system('git clone https://github.com/alexgkendall/SegNet-Tutorial ./data')
    print('Done!')

In [ ]:
x_train_dir = os.path.join(DATA_DIR, 'train')
y_train_dir = os.path.join(DATA_DIR, 'trainannot')

x_valid_dir = os.path.join(DATA_DIR, 'val')
y_valid_dir = os.path.join(DATA_DIR, 'valannot')

x_test_dir = os.path.join(DATA_DIR, 'test')
y_test_dir = os.path.join(DATA_DIR, 'testannot')

In [ ]:
class Dataset(object):
    """The base class for dataset classes.
    To use it, create a new class that adds functions specific to the dataset
    you want to use. For example:

    class CatsAndDogsDataset(Dataset):
        def load_cats_and_dogs(self):
            ...
        def load_mask(self, image_id):
            ...
        def image_reference(self, image_id):
            ...

    See COCODataset and ShapesDataset as examples.
    """

    def __init__(self, class_map=None):
        self._image_ids = []
        self.image_info = []
        # Background is always the first class
        self.class_info = [{"source": "", "id": 0, "name": "BG"}]
        self.source_class_ids = {}

    def add_class(self, source, class_id, class_name):
        assert "." not in source, "Source name cannot contain a dot"
        # Does the class exist already?
        for info in self.class_info:
            if info['source'] == source and info["id"] == class_id:
                # source.class_id combination already available, skip
                return
        # Add the class
        self.class_info.append({
            "source": source,
            "id": class_id,
            "name": class_name,
        })

    def add_image(self, source, image_id, path, **kwargs):
        image_info = {
            "id": image_id,
            "source": source,
            "path": path,
        }
        image_info.update(kwargs)
        self.image_info.append(image_info)

    def image_reference(self, image_id):
        """Return a link to the image in its source Website or details about
        the image that help looking it up or debugging it.

        Override for your dataset, but pass to this function
        if you encounter images not in your dataset.
        """
        return ""

    def prepare(self, class_map=None):
        """Prepares the Dataset class for use.

        TODO: class map is not supported yet. When done, it should handle mapping
              classes from different datasets to the same class ID.
        """

        def clean_name(name):
            """Returns a shorter version of object names for cleaner display."""
            return ",".join(name.split(",")[:1])

        # Build (or rebuild) everything else from the info dicts.
        self.num_classes = len(self.class_info)
        print("Num Class",self.num_classes)
        self.class_ids = np.arange(self.num_classes)
        self.class_names = [clean_name(c["name"]) for c in self.class_info]
        self.num_images = len(self.image_info)
        self._image_ids = np.arange(self.num_images)

        # Mapping from source class and image IDs to internal IDs
        self.class_from_source_map = {"{}.{}".format(info['source'], info['id']): id
                                      for info, id in zip(self.class_info, self.class_ids)}
        self.image_from_source_map = {"{}.{}".format(info['source'], info['id']): id
                                      for info, id in zip(self.image_info, self.image_ids)}

        # Map sources to class_ids they support
        self.sources = list(set([i['source'] for i in self.class_info]))
        self.source_class_ids = {}
        # Loop over datasets
        for source in self.sources:
            self.source_class_ids[source] = []
            # Find classes that belong to this dataset
            for i, info in enumerate(self.class_info):
                # Include BG class in all datasets
                if i == 0 or source == info['source']:
                    self.source_class_ids[source].append(i)

    def map_source_class_id(self, source_class_id):
        """Takes a source class ID and returns the int class ID assigned to it.

        For example:
        dataset.map_source_class_id("coco.12") -> 23
        """
        return self.class_from_source_map[source_class_id]

    def get_source_class_id(self, class_id, source):
        """Map an internal class ID to the corresponding class ID in the source dataset."""
        info = self.class_info[class_id]
        assert info['source'] == source
        return info['id']

    @property
    def image_ids(self):
        return self._image_ids

    def source_image_link(self, image_id):
        """Returns the path or URL to the image.
        Override this to return a URL to the image if it's available online for easy
        debugging.
        """
        return self.image_info[image_id]["path"]

    def load_image(self, image_id):
        """Load the specified image and return a [H,W,3] Numpy array.
        """
        # Load image
        image = skimage.io.imread(self.image_info[image_id]['path'])
        # If grayscale. Convert to RGB for consistency.
        if image.ndim != 3:
            image = skimage.color.gray2rgb(image)
        # If has an alpha channel, remove it for consistency
        if image.shape[-1] == 4:
            image = image[..., :3]
        return image

    def load_mask(self, image_id):
        """Load instance masks for the given image.

        Different datasets use different ways to store masks. Override this
        method to load instance masks and return them in the form of am
        array of binary masks of shape [height, width, instances].

        Returns:
            masks: A bool array of shape [height, width, instance count] with
                a binary mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        # Override this function to load a mask from your dataset.
        # Otherwise, it returns an empty mask.
        logging.warning("You are using the default load_mask(), maybe you need to define your own one.")
        mask = np.empty([0, 0, 0])
        class_ids = np.empty([0], np.int32)
        return mask, class_ids
    
# helper function for data visualization
def visualize_unet(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(30, 10))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x
    

# classes for data loading and preprocessing
class CorkDatasetUNet(Dataset):
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    
    
    def __init__(
            self, 
            images_dir,
            annot_dir,
            augmentation=None, 
            preprocessing=None,
    ):
        Dataset.__init__(self)
        #self.ids = os.listdir(images_dir)
        #self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.ids=[]
        self.images_fps=[]
        #self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # convert str names to class values on masks
        #self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]

        self.add_class("Wood", 1, "Wood")

        # Train or validation dataset?
        #assert subset in ["train", "val"]
        #dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator (up to version 1.6) saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        # Note: In VIA 2.0, regions was changed from a dict to a list.
        annotations = json.load(open(os.path.join(annot_dir, "via_region_data.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        #annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. These are stores in the
            # shape_attributes (see json format above)
            # The if condition is needed to support VIA versions 1.x and 2.x.
            if type(a['regions']) is dict:
                polygons = [r['shape_attributes'] for r in a['regions'].values()]
            else:
                polygons = [r['shape_attributes'] for r in a['regions']] 

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            self.ids.append(a['filename'])
            image_path = os.path.join(images_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]
            self.images_fps.append(image_path)
            
            self.add_image(
                "Wood",
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons)


        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    
    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a cork dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "Wood":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.int), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "Wood":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #mask = cv2.imread(self.masks_fps[i], 0)
        #
        #print(mask.shape)
        ##print("Class Val",self.class_values)
        ##print("Mask Shape", mask.shape)
        ## extract certain classes from mask (e.g. cars)
        #masks = [(mask == v) for v in self.class_values]
        ##print("Masks---", masks)
        #mask = np.stack(masks, axis=-1).astype('float')
        #print("After Stack",mask)

        mask, class_ids = self.load_mask(i)
        mask=np.sum(mask,axis=2)
        mask=np.expand_dims(mask,axis=2)

        assert np.any(mask) <= 1, "Overlapping annotations are there!"
        mask = mask.astype(float)


        # add background if mask is not binary
        if mask.shape[-1] == 1:
            background = 1 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        return len(self.ids)
    
    
class Dataloder(keras.utils.Sequence):
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):
        
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
        
        # transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        
        return tuple(batch)
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)
            
def round_clip_0_1(x, **kwargs):
    return x.round().clip(0, 1)

# define heavy augmentations
def get_training_augmentation():
    train_transform = [

        A.HorizontalFlip(p=0.5),

        #A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        #A.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
        A.RandomCrop(height=640, width=640, always_apply=True),

        #A.GaussNoise(p=0.2),
        #A.Perspective(p=0.5),

        #A.OneOf(
        #    [
        #        A.CLAHE(p=1),
        #        A.RandomBrightness(p=1),
        #        A.RandomGamma(p=1),
        #    ],
        #    p=0.9,
        #),

        #A.OneOf(
        #    [
        #        A.Sharpen(p=1),
        #        A.Blur(blur_limit=3, p=1),
        #        A.MotionBlur(blur_limit=3, p=1),
        #    ],
        #    p=0.9,
        #),

        #A.OneOf(
        #    [
        #        A.RandomBrightnessContrast(p=1),
        #        A.HueSaturationValue(p=1),
        #    ],
        #    p=0.9,
        #),
        #A.Lambda(mask=round_clip_0_1)
    ]
    return A.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        A.PadIfNeeded(1504, 1984)
    ]
    return A.Compose(test_transform)

def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        A.Lambda(image=preprocessing_fn),
    ]
    return A.Compose(_transform)

In [ ]:
BACKBONE = 'efficientnetb3'
BATCH_SIZE = 8
CLASSES = ['Wood']
LR = 0.0001
EPOCHS = 50

preprocess_input = sm.get_preprocessing(BACKBONE)

n_classes = 2
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model
model_unet = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

# define optomizer
optim = keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimozer, loss and metrics
model_unet.compile(optim, total_loss, metrics)

test_dataset = CorkDatasetUNet(
    x_valid_dir, 
    y_valid_dir, 
    augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocess_input),
)

test_dataloader = Dataloder(test_dataset, batch_size=1, shuffle=False)

In [ ]:
# load best weights
model_unet.load_weights('../segmentation_models/1_ENetB3_lr0001_bs8_epo100_softmax.h5')
n = 20
ids = np.arange(len(test_dataset))

#np.random.choice

for i in ids:
    
    image, gt_mask = test_dataset[i]
    image = np.expand_dims(image, axis=0)
    t = time.time()
    pr_mask = model_unet.predict(image).round()
    pred_time = time.time() - t
    print("Eval Time-",pred_time)
    print("UNET MASK SHAPE",pr_mask.shape)
    visualize_unet(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[..., 0].squeeze(),
        pr_mask=pr_mask[..., 0].squeeze(),
        #bg_mask=bg_mask[..., 1].squeeze()
    )

# Ensemble 

### Important Functions

In [ ]:
def padIfNeeded(img, h, w):
    delta_w = w - img.shape[1]
    delta_h = h - img.shape[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    #color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT)
    
    return new_im

def iou_score(mask1, mask2):
    """Computes IoU overlaps between two sets of masks.
    masks1, masks2: [Height, Width, instances]
    """    

    # intersections and union
    intersection = np.sum((mask1 + mask2)==2)
    union = np.sum(mask1) + np.sum(mask2) - intersection
    if union==0:
        return 1
    overlaps = intersection / union

    return overlaps


    

def ensemble_type1(img1,img2, rcnn_scores):
    
    ch_1 = img1.shape[2]
    try:
        ch_2 = img2.shape[2]
    except:
        return img1
    
    mask=[]
    i_unet_ind=np.zeros(ch_1)
    j_rcnn_ind=np.zeros(ch_2)
    for i in range(ch_1):
        for j in range(ch_2):
            if np.any(img1[:,:,i]+img2[:,:,j]==2):
                m=np.where(img1[:,:,i]+img2[:,:,j]==2,1,0)
                mask.append(m)
                i_unet_ind[i]=1
                j_rcnn_ind[j]=1
            
        #if vis==0:
        #    mask.append(img1[:,:,i])
        #    mask.append(img2[:,:,j])
        
    for i in range(ch_1):
        if i_unet_ind[i] == 0:
            area=utils.compute_mask_area(np.expand_dims(img1[:,:,i],axis=2))
            if area > 500:
                mask.append(img1[:,:,i])
    
    for j in range(ch_2):
        if j_rcnn_ind[j] == 0:
            area=utils.compute_mask_area(np.expand_dims(img2[:,:,j],axis=2))
            if area > 500 and rcnn_scores[j] > 0.95:
                mask.append(img2[:,:,j])

    try:
        mask_ensemble=np.stack(mask,axis=2)
    except:
        return img1
    return mask_ensemble
    
#image = dataset_val.load_image(0)
#new_img = padIfNeeded(image, 1504, 1984)
#plt.imshow(new_img)
#new_img.shape

def bin_mask_splitter(im):
    image=im.squeeze()
    labeled = morph.label(image, connectivity=2)

    mask=[]
    for i in np.unique(labeled)[1:]: # skip the first component since it's the background
        im_obj = np.zeros(image.shape) 
        im_obj[labeled == i] = 1
        #imsave('sLUel_{:03d}.png'.format(i), im_obj)
        mask.append(im_obj)
    
    if not len(mask):
        return im
    final=np.stack(mask,axis=2)
    return final

msk = []
mask_unet1 = np.array([[0,0,0,0,0,0],[0,1,1,0,0,0],[0,0,1,1,0,0],[0,0,0,0,0,0]])
mask_unet2 = np.array([[0,0,0,0,0,1],[0,0,0,0,1,1],[0,0,0,0,0,1],[0,0,0,0,0,0]])
msk.append(mask_unet1)
msk.append(mask_unet2)
mask_unet=np.stack(msk,axis=2)

msk=[]
mask_rcnn1 = np.array([[0,1,1,0,0,0],[0,1,1,0,0,0],[0,0,1,1,0,0],[0,0,1,1,0,0]])
mask_rcnn2 = np.array([[0,0,0,0,1,1],[0,0,0,0,1,1],[0,0,0,0,0,0],[0,0,0,0,0,0]])
mask_rcnn3 = np.array([[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,1,1]])
msk.append(mask_rcnn1)
msk.append(mask_rcnn2)
msk.append(mask_rcnn3)
mask_rcnn=np.stack(msk,axis=2)



#m=np.where(mask_rcnn2==mask_unet2,mask_rcnn2,0)
m=padIfNeeded(mask_unet,20,20)

plt.imshow(m[:,:,1])


In [ ]:
a=np.zeros(4)

In [ ]:
p=np.array([-1,-1,3,2,-1])
#np.cumsum(p > -1)

#(np.arange(len(p)) + 1)

mAP(mask_rcnn, mask_unet)

# Visual Comparison: U-Net vs Mask R-CNN

In [ ]:
ids = np.arange(len(test_dataset))

AP_lst=[]
IoU_mean=[]
AP_ensLst = []

#ids = (15,16)

for i in ids:
    
    image = dataset_val.load_image(i)

    config.IMAGE_RESIZE_MODE="none"
    orig_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, config,i)
    
    #image_rcnn = image.astype(np.uint8)
    #image = dataset_val.load_image(i)
    #print("MRNN DTP",image.dtype)
    config.IMAGE_RESIZE_MODE="square"
    orig_image_vis, image_meta_vis, gt_class_id_vis, gt_bbox_vis, gt_mask_vis =\
        modellib.load_image_gt(dataset_val, config,i)
    visualize.display_instances(orig_image_vis, gt_bbox_vis, gt_mask_vis, gt_class_id_vis, dataset_val.class_names, title="GT", figsize=(8, 8))
        
    
    t = time.time()
    r = model_mrcnn.detect([image], verbose=0)[0]
    
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], title=fnameP, ax=get_ax())

    AP_x, precisions_x, recalls_x, overlaps_x =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    print("Mask RCNN mAP", AP_x)
    
    rcnn_mask = r['masks'].astype(np.int32)
    print("RCNN Mask",rcnn_mask.shape)
    
    gt_mask=gt_mask.astype(np.int32)
    
    #For shape mismatch in 1 layer Mask -> [[...]]
    if gt_mask.shape[2] == 1:
        gt_mask=padIfNeeded(gt_mask, 1504, 1984)
        gt_mask=np.expand_dims(gt_mask,axis=2)
    #For shape mismatch in empty Mask -> [[]]    
    elif gt_mask.shape[2]==0:
        if rcnn_mask.shape[2]==0:
            print("Image Number----------", i)
            print("maskrcnn type-",rcnn_mask.dtype)
            print("maskrcnn GT Mask TYPE",gt_mask.dtype)
            print("unet type-",unet_mask.dtype)
            print("maskrcnn Pred mask shape-",rcnn_mask.shape)
            print("maskrcnn GT Mask shape",gt_mask.shape)
            print("unet mask shape-",unet_mask.shape)
            print("---------------------------")
            AP_ensLst.append(1.0)
            IoU_mean.append(1.0)
            continue
        else:
            AP_ensLst.append(0.0)
            IoU_mean.append(0.0)
            continue
    else:           
        gt_mask=padIfNeeded(gt_mask, 1504, 1984)
    
    
    
    rcnn_mask=padIfNeeded(rcnn_mask, 1504, 1984)
    
    
    #UNET mask calculation
    image, gt_mask_unet = test_dataset[i]
    
    #print("UNET DTP",image.dtype)
    image = np.expand_dims(image, axis=0)
    unet_mask = model_unet.predict(image).round()
    
    unet_mask = unet_mask.squeeze()
    unet_mask_b4split = unet_mask[:,:,0:1]
    
    visualize_unet(
       unet_mask=unet_mask_b4split[..., 0].squeeze(),
        #bg_mask=bg_mask[..., 1].squeeze()
    )
    
    unet_mask=bin_mask_splitter(unet_mask_b4split)
    unet_mask=unet_mask.astype(np.int32)
    
    
    
    print("======================================")
    print("Image Number----------", i)
    print("maskrcnn type-",rcnn_mask.dtype)
    print("maskrcnn GT Mask TYPE",gt_mask.dtype)
    print("unet type-",unet_mask.dtype)
    print("maskrcnn Pred mask shape-",rcnn_mask.shape)
    print("maskrcnn GT Mask shape",gt_mask.shape)
    print("unet mask shape-",unet_mask.shape)
    print("---------------------------")
    
    AP=mAP(gt_mask,unet_mask)
    print("AP_Unet",AP)
    AP_lst.append(AP)
    
    output_mask=ensemble_type1(unet_mask,rcnn_mask, r['scores'])
    t_rcnn=time.time()-t
    print("Pred Time",t_rcnn)
    iou=utils.compute_overlaps_masks(gt_mask,output_mask)
    if len(iou)==0:
        iou=0
    else:
        iou=np.sum(iou,axis=1)
        iou=np.mean(iou)
        
    IoU_mean.append(iou)
    print("IoU Ind mean",iou)
    print("MASK AREA", utils.compute_mask_area(output_mask))
    print("ensemble mask shape-",output_mask.shape)
    vmask=np.sum(output_mask,axis=2)
    vmask=np.expand_dims(vmask,axis=2)
    visualize_unet(
        pr_mask=vmask[..., 0].squeeze(),
        #bg_mask=bg_mask[..., 1].squeeze()
    )
    AP=mAP(gt_mask,output_mask,0.7)
    print("AP_ensemble",AP)
    AP_ensLst.append(AP)
    print("===============================")
    
print("MEAN AP of Unet", np.mean(AP_lst))
print("MEAN AP of Ensemble", np.mean(AP_ensLst))
print("MEAN IOU of Ensemble", np.mean(IoU_mean))
    
    

# Weighted Average Ensemble

## Important Functions

In [ ]:
from itertools import product
from numpy.linalg import norm
 
def ensemble_prediction(w):
    ids = np.arange(len(test_dataset))

    avg_iou_rcnn=[]
    avg_iou_unet=[]
    avg_iou_ens=[]
    
    avg_dice_rcnn=[]
    avg_dice_unet=[]
    avg_dice_ens=[]
    
    
    #ids = (8,12,15,16)
    
    for i in ids:
        
        image = dataset_val.load_image(i)
    
        config.IMAGE_RESIZE_MODE="none"
        orig_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_val, config,i)
        
        #image_rcnn = image.astype(np.uint8)
        #image = dataset_val.load_image(i)
        #print("MRNN DTP",image.dtype)
        config.IMAGE_RESIZE_MODE="square"
        #orig_image_vis, image_meta_vis, gt_class_id_vis, gt_bbox_vis, gt_mask_vis =\
        #    modellib.load_image_gt(dataset_val, config,i)
        #visualize.display_instances(orig_image_vis, gt_bbox_vis, gt_mask_vis, gt_class_id_vis, dataset_val.class_names, title="GT", figsize=(8, 8))
        
        t = time.time()
        r = model_mrcnn.detect([image], verbose=0)[0]
        
        #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
        #                        dataset_val.class_names, r['scores'], title=fnameP, ax=get_ax())
    
        threshold=0.5
        res_mask=r['masks']
        res_mask=np.where(res_mask >= threshold, 1, 0).astype(np.bool)
        #AP_x, precisions_x, recalls_x, overlaps_x =\
        #    utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
        #                     r["rois"], r["class_ids"], r["scores"], res_mask)
        
        
        #print("RCNN Mask shape b4 summation",r['masks'].shape)
        
        
        #### Converting the gt and pred mask into binary semantic segmentation ###
        gt_mask_4unet=gt_mask
        gt_mask = np.sum(gt_mask,axis=2)
        gt_mask = np.expand_dims(gt_mask,axis=2)
                
        rcnn_mask = np.sum(r['masks'],axis=2)
        #rcnn_mask = np.where(rcnn_mask>1.0,1.0,rcnn_mask)
        rcnn_mask = np.expand_dims(rcnn_mask,axis=2)
        
        rmask=np.sum(res_mask,axis=2)
        #rcnn_mask = np.where(rcnn_mask>1.0,1.0,rcnn_mask)
        rmask = np.expand_dims(rmask,axis=2)
        
        
        #print("GT Mask shape after summation",gt_mask.dtype)
        #print("RCNN Mask shape after summation",rmask.dtype)
        IoU_rcnn=iou_score(rmask,gt_mask)
        d_score=dice_score(rmask,gt_mask)
        #print("mAP MRCNN", AP_x)
        #print("IoU Score MRCNN", IoU_rcnn)
        #print("DICE MRCNN",d_score)
        #print("===========================")
        avg_iou_rcnn.append(IoU_rcnn)
        avg_dice_rcnn.append(d_score)
        
        
        rcnn_mask=padIfNeeded(rcnn_mask, 1504, 1984)
        rcnn_mask=np.expand_dims(rcnn_mask,axis=2)
        #print("RCNN Mask shape after padding and 2 times expndDims",rcnn_mask.shape)
        
        #visualize_unet(
        #   predrcnn_msk=rcnn_mask[..., 0],
            #bg_mask=bg_mask[..., 1].squeeze()
        #)
        
        ## UNET MODEL as a Base Learner #########
        image, gt_mask_unet = test_dataset[i]
        
        gt_mask_unet=gt_mask_unet[:,:,0:1]
        
        
        image = np.expand_dims(image, axis=0)
        unet_mask = model_unet.predict(image)
        
        unet_mask=unet_mask.squeeze()
        unet_mask=unet_mask[:,:,0:1]
        
        #print("UNET Mask shape",unet_mask.shape)
        #print("GT unet Mask shape",gt_mask_unet.shape)
        
        iou_unet=iou_score(gt_mask_unet,unet_mask.round())
        d_score=dice_score(gt_mask_unet.astype(int),unet_mask.round())  
        
        #visualize_unet(
        #   unet_mask=unet_mask[..., 0],
            #bg_mask=bg_mask[..., 1].squeeze()
        #)
        
        #unet_mask_inst=bin_mask_splitter(unet_mask.round())
        #gt_mask_inst=bin_mask_splitter(gt_mask_unet)
        
            
        
        #print("UNET Mask shape after morph",unet_mask_inst.dtype)
        #print("GT unet Mask shape after morph",gt_mask_inst.shape)
        
        #mAP_val=mAP(gt_mask_inst,unet_mask_inst)
        #print("IoU Unet",iou_unet)
        #print("DICE UNET",d_score)
        #print("===========================")
        #print("MAP UNET",mAP_val)
        avg_iou_unet.append(iou_unet)
        avg_dice_unet.append(d_score)
        
        
        ##### Weighted Average Ensemble ######################
        w = normalize(w)
        ensemble_mask=w[0]*unet_mask+w[1]*rcnn_mask
        ensemble_mask=ensemble_mask.round()
        
        #visualize_unet(
        #   ensemble_mask=ensemble_mask[..., 0],
            #bg_mask=bg_mask[..., 1].squeeze()
        #)
        
        iou_ensemble=iou_score(gt_mask_unet,ensemble_mask)
        d_score=dice_score(gt_mask_unet,ensemble_mask)
        #print("ENSEMBLE IOU SCORE",iou_ensemble)
        #print("ENSEMBLE DICE",d_score)
        #print("==========================")
        avg_iou_ens.append(iou_ensemble)
        avg_dice_ens.append(d_score)
        t_total=time.time()-t
        #print("Pred Time",t_total)
        
        
    return 1-np.mean(avg_iou_ens)

def normalize(weights):
	# calculate l1 vector norm
	result = norm(weights, 1)
	# check for a vector of all zeros
	if result == 0.0:
		return weights
	# return normalized vector (unit norm)
	return weights / result


def grid_search():
    # define weights to consider
    w = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    best_score, best_weights = 0.0, None
    # iterate all possible combinations (cartesian product)
    for weights in product(w, repeat=2):
        # skip if all weights are equal
        if len(set(weights)) == 1:
            continue
        # hack, normalize weight vector
        weights = normalize(weights)
        # evaluate weights
        score = ensemble_prediction(weights)
        print("Output for:",weights)
        if score > best_score:
            best_score, best_weights = score, weights
            print('>%s %.3f' % (best_weights, best_score))
    return list(best_weights)

In [ ]:
ids = np.arange(len(test_dataset))

avg_iou_rcnn=[]
avg_iou_unet=[]
avg_iou_ens=[]

avg_dice_rcnn=[]
avg_dice_unet=[]
avg_dice_ens=[]

avg_prec_unet=[]
avg_rec_unet=[]


avg_prec=[]
avg_rec=[]

map_rcnn=[]
map_unet=[]
map_ens=[]

#ids = (8,12,15,16)

for i in ids:
    
    image = dataset_val.load_image(i)

    config.IMAGE_RESIZE_MODE="none"
    orig_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, config,i)
    
    #image_rcnn = image.astype(np.uint8)
    #image = dataset_val.load_image(i)
    #print("MRNN DTP",image.dtype)
    config.IMAGE_RESIZE_MODE="square"
    orig_image_vis, image_meta_vis, gt_class_id_vis, gt_bbox_vis, gt_mask_vis =\
        modellib.load_image_gt(dataset_val, config,i)
    visualize.display_instances(orig_image_vis, gt_bbox_vis, gt_mask_vis, gt_class_id_vis, dataset_val.class_names, title="GT", figsize=(8, 8))
    
    t = time.time()
    r = model_mrcnn.detect([image], verbose=0)[0]
    
    threshold=0.5
    res_mask=r['masks']
    res_mask=np.where(res_mask >= threshold, 1, 0).astype(np.bool)
    
    visualize.display_instances(image, r['rois'], res_mask, r['class_ids'], 
                            dataset_val.class_names, r['scores'], title=fnameP, ax=get_ax())

    
    AP_x, precisions_x, recalls_x, overlaps_x =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], res_mask)
    
    
    #print("RCNN Mask shape b4 summation",r['masks'].shape)
    
    
    #### Converting the gt and pred mask into binary semantic segmentation ###
    gt_mask_4unet=gt_mask
    gt_mask = np.sum(gt_mask,axis=2)
    gt_mask = np.expand_dims(gt_mask,axis=2)
            
    visualize_unet(
       gt_mask=gt_mask[..., 0],
        #bg_mask=bg_mask[..., 1].squeeze()
    ) 
    
    rcnn_mask = np.sum(r['masks'],axis=2)
    #rcnn_mask = np.where(rcnn_mask>1.0,1.0,rcnn_mask)
    rcnn_mask = np.expand_dims(rcnn_mask,axis=2)
    
    rmask=np.sum(res_mask,axis=2)
    #rcnn_mask = np.where(rcnn_mask>1.0,1.0,rcnn_mask)
    rmask = np.expand_dims(rmask,axis=2)
    
    
    print("GT Mask shape after summation",gt_mask.dtype)
    print("RCNN Mask shape after summation",rmask.dtype)
    IoU_rcnn=iou_score(rmask,gt_mask)
    d_score=dice_score(rmask,gt_mask)
    map_maskrcnn=map_t(gt_mask,rmask)
    #print("mAP MRCNN", AP_x)
    print("IoU Score MRCNN", IoU_rcnn)
    print("DICE MRCNN",d_score)
    print("MAP rcnn",map_maskrcnn)
    print("===========================")
    avg_iou_rcnn.append(IoU_rcnn)
    avg_dice_rcnn.append(d_score)
    map_rcnn.append(map_maskrcnn)
    
    rcnn_mask=padIfNeeded(rcnn_mask, 1504, 1984)
    rcnn_mask=np.expand_dims(rcnn_mask,axis=2)
    #print("RCNN Mask shape after padding and 2 times expndDims",rcnn_mask.shape)
    
    visualize_unet(
       predrcnn_msk=rcnn_mask[..., 0].round(),
        #bg_mask=bg_mask[..., 1].squeeze()
    )
    
    ## UNET MODEL as a Base Learner #########
    image, gt_mask_unet = test_dataset[i]
    
    gt_mask_unet=gt_mask_unet[:,:,0:1]
    
    
    image = np.expand_dims(image, axis=0)
    unet_mask = model_unet.predict(image)
    
    unet_mask=unet_mask.squeeze()
    unet_mask=unet_mask[:,:,0:1]
    
    #print("UNET Mask shape",unet_mask.shape)
    #print("GT unet Mask shape",gt_mask_unet.shape)
    
    iou_unet=iou_score(gt_mask_unet,unet_mask.round())
    d_score=dice_score(gt_mask_unet.astype(int),unet_mask.round())  
    prec,recall=precision_recall(gt_mask_unet,unet_mask.round())
    map_unett=map_t(gt_mask_unet,unet_mask.round())
    
    visualize_unet(
       unet_mask=unet_mask[..., 0].round(),
        #bg_mask=bg_mask[..., 1].squeeze()
    )
    
    #unet_mask_inst=bin_mask_splitter(unet_mask.round())
    #gt_mask_inst=bin_mask_splitter(gt_mask_unet)
    
        
    
    #print("UNET Mask shape after morph",unet_mask_inst.dtype)
    #print("GT unet Mask shape after morph",gt_mask_inst.shape)
    
    #mAP_val=mAP(gt_mask_inst,unet_mask_inst)
    print("IoU Unet",iou_unet)
    print("DICE UNET",d_score)
    print("Prec UNET",prec)
    print("Rec Unet",recall)
    print("mAP UNET",map_unett)
    print("===========================")
    #print("MAP UNET",mAP_val)
    avg_iou_unet.append(iou_unet)
    avg_dice_unet.append(d_score)
    avg_prec_unet.append(prec)
    avg_rec_unet.append(recall)
    map_unet.append(map_unett)
    
    ##### Weighted Average Ensemble ######################
    ##0.625 0.375
    ensemble_mask=0.625*unet_mask+0.375*rcnn_mask
    ensemble_mask=ensemble_mask.round()
    
    visualize_unet(
       ensemble_mask=ensemble_mask[..., 0],
        #bg_mask=bg_mask[..., 1].squeeze()
    )
    
    iou_ensemble=iou_score(gt_mask_unet,ensemble_mask)
    d_score=dice_score(gt_mask_unet,ensemble_mask)
    prec,recall=precision_recall(gt_mask_unet,ensemble_mask)
    map_enss=map_t(gt_mask_unet,ensemble_mask)
    print("ENSEMBLE IOU SCORE",iou_ensemble)
    print("ENSEMBLE DICE",d_score)
    print("Ensemble Prec", prec)
    print("Ensemble Recall", recall)
    print("Ensemble MAP",map_enss)
    map_ens.append(map_enss)
    if not np.any(gt_mask_unet) and not np.any(ensemble_mask):
        print("GOOD CORK")
        avg_iou_ens.append(1)
        avg_dice_ens.append(1)
        avg_prec.append(1)
        avg_rec.append(1)
    else:
        
        avg_iou_ens.append(iou_ensemble)
        avg_dice_ens.append(d_score)
        avg_prec.append(prec)
        avg_rec.append(recall)
    print("==========================")
    t_total=time.time()-t
    print("Pred Time",t_total)
    
    
print("IOU MRCNN", np.mean(avg_iou_rcnn))
print("IOU UNET", np.mean(avg_iou_unet))
print("IOU of Ensemble", np.mean(avg_iou_ens))

    
print("Dice MRCNN", np.mean(avg_dice_rcnn))
print("Dice UNET", np.mean(avg_dice_unet))
print("Dice of Ensemble", np.mean(avg_dice_ens))
    
print("Prec UNET", np.mean(avg_prec_unet))
print("Recall UNET", np.mean(avg_rec_unet))
print("Prec of Ensemble", np.mean(avg_prec))
print("Recall of Ensemble", np.mean(avg_rec)) 

print("map MRCNN", np.mean(map_rcnn))
print("map UNET", np.mean(map_unet))
print("map of Ensemble", np.mean(map_ens))

    

# Ensemble PloT

In [ ]:
iou_t=[0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95]
rcnn_iou=[]
unet_iou=[]
ens_iou=[]
for th in iou_t:	
	ids = np.arange(len(test_dataset))
	
	avg_iou_rcnn=[]
	avg_iou_unet=[]
	avg_iou_ens=[]
	
	avg_dice_rcnn=[]
	avg_dice_unet=[]
	avg_dice_ens=[]
	
	avg_prec_unet=[]
	avg_rec_unet=[]
	
	
	avg_prec=[]
	avg_rec=[]
	
	map_rcnn=[]
	map_unet=[]
	map_ens=[]
	
	#ids = (8,12,15,16)
	
	for i in ids:
		
		image = dataset_val.load_image(i)
	
		config.IMAGE_RESIZE_MODE="none"
		orig_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
			modellib.load_image_gt(dataset_val, config,i)
		
		#image_rcnn = image.astype(np.uint8)
		#image = dataset_val.load_image(i)
		#print("MRNN DTP",image.dtype)
		config.IMAGE_RESIZE_MODE="square"
		orig_image_vis, image_meta_vis, gt_class_id_vis, gt_bbox_vis, gt_mask_vis =\
			modellib.load_image_gt(dataset_val, config,i)
		#visualize.display_instances(orig_image_vis, gt_bbox_vis, gt_mask_vis, gt_class_id_vis, dataset_val.class_names, title="GT", figsize=(8, 8))
		
		t = time.time()
		r = model_mrcnn.detect([image], verbose=0)[0]
		
		#visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
		#                        dataset_val.class_names, r['scores'], title=fnameP, ax=get_ax())
	
		threshold=0.5
		res_mask=r['masks']
		res_mask=np.where(res_mask >= threshold, 1, 0).astype(np.bool)
		AP_x, precisions_x, recalls_x, overlaps_x =\
			utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
							r["rois"], r["class_ids"], r["scores"], res_mask)
		
		
		#print("RCNN Mask shape b4 summation",r['masks'].shape)
		
		
		#### Converting the gt and pred mask into binary semantic segmentation ###
		gt_mask_4unet=gt_mask
		gt_mask = np.sum(gt_mask,axis=2)
		gt_mask = np.expand_dims(gt_mask,axis=2)
				
		visualize_unet(
		gt_mask=gt_mask[..., 0],
			#bg_mask=bg_mask[..., 1].squeeze()
		) 
		
		rcnn_mask = np.sum(r['masks'],axis=2)
		#rcnn_mask = np.where(rcnn_mask>1.0,1.0,rcnn_mask)
		rcnn_mask = np.expand_dims(rcnn_mask,axis=2)
		
		rmask=np.sum(res_mask,axis=2)
		#rcnn_mask = np.where(rcnn_mask>1.0,1.0,rcnn_mask)
		rmask = np.expand_dims(rmask,axis=2)
		
		
		print("GT Mask shape after summation",gt_mask.dtype)
		print("RCNN Mask shape after summation",rmask.dtype)
		IoU_rcnn=iou_score(rmask,gt_mask)
		d_score=dice_score(rmask,gt_mask)
		map_maskrcnn=map_t(gt_mask,rmask,th)
		#print("mAP MRCNN", AP_x)
		print("IoU Score MRCNN", IoU_rcnn)
		print("DICE MRCNN",d_score)
		print("MAP rcnn",map_maskrcnn)
		print("===========================")
		avg_iou_rcnn.append(IoU_rcnn)
		avg_dice_rcnn.append(d_score)
		map_rcnn.append(map_maskrcnn)
		
		rcnn_mask=padIfNeeded(rcnn_mask, 1504, 1984)
		rcnn_mask=np.expand_dims(rcnn_mask,axis=2)
		#print("RCNN Mask shape after padding and 2 times expndDims",rcnn_mask.shape)
		
		visualize_unet(
		predrcnn_msk=rcnn_mask[..., 0].round(),
			#bg_mask=bg_mask[..., 1].squeeze()
		)
		
		## UNET MODEL as a Base Learner #########
		image, gt_mask_unet = test_dataset[i]
		
		gt_mask_unet=gt_mask_unet[:,:,0:1]
		
		
		image = np.expand_dims(image, axis=0)
		unet_mask = model_unet.predict(image)
		
		unet_mask=unet_mask.squeeze()
		unet_mask=unet_mask[:,:,0:1]
		
		#print("UNET Mask shape",unet_mask.shape)
		#print("GT unet Mask shape",gt_mask_unet.shape)
		
		iou_unet=iou_score(gt_mask_unet,unet_mask.round())
		d_score=dice_score(gt_mask_unet.astype(int),unet_mask.round())  
		prec,recall=precision_recall(gt_mask_unet,unet_mask.round())
		map_unett=map_t(gt_mask_unet,unet_mask.round(),th)
		
		visualize_unet(
		unet_mask=unet_mask[..., 0].round(),
			#bg_mask=bg_mask[..., 1].squeeze()
		)
		
		#unet_mask_inst=bin_mask_splitter(unet_mask.round())
		#gt_mask_inst=bin_mask_splitter(gt_mask_unet)
		
			
		
		#print("UNET Mask shape after morph",unet_mask_inst.dtype)
		#print("GT unet Mask shape after morph",gt_mask_inst.shape)
		
		#mAP_val=mAP(gt_mask_inst,unet_mask_inst)
		print("IoU Unet",iou_unet)
		print("DICE UNET",d_score)
		print("Prec UNET",prec)
		print("Rec Unet",recall)
		print("mAP UNET",map_unett)
		print("===========================")
		#print("MAP UNET",mAP_val)
		avg_iou_unet.append(iou_unet)
		avg_dice_unet.append(d_score)
		avg_prec_unet.append(prec)
		avg_rec_unet.append(recall)
		map_unet.append(map_unett)
		
		##### Weighted Average Ensemble ######################
		##0.625 0.375
		ensemble_mask=0.625*unet_mask+0.375*rcnn_mask
		ensemble_mask=ensemble_mask.round()
		
		visualize_unet(
		ensemble_mask=ensemble_mask[..., 0],
			#bg_mask=bg_mask[..., 1].squeeze()
		)
		
		iou_ensemble=iou_score(gt_mask_unet,ensemble_mask)
		d_score=dice_score(gt_mask_unet,ensemble_mask)
		prec,recall=precision_recall(gt_mask_unet,ensemble_mask)
		map_enss=map_t(gt_mask_unet,ensemble_mask,th)
		print("ENSEMBLE IOU SCORE",iou_ensemble)
		print("ENSEMBLE DICE",d_score)
		print("Ensemble Prec", prec)
		print("Ensemble Recall", recall)
		print("Ensemble MAP",map_enss)
		map_ens.append(map_enss)
		if not np.any(gt_mask_unet) and not np.any(ensemble_mask):
			print("GOOD CORK")
			avg_iou_ens.append(1)
			avg_dice_ens.append(1)
			avg_prec.append(1)
			avg_rec.append(1)
		else:
			
			avg_iou_ens.append(iou_ensemble)
			avg_dice_ens.append(d_score)
			avg_prec.append(prec)
			avg_rec.append(recall)
		print("==========================")
		t_total=time.time()-t
		print("Pred Time",t_total)
		
	print("IOU threshold",th)
	rcnn_iou.append(np.mean(map_rcnn))
	unet_iou.append(np.mean(map_unet))
	ens_iou.append(np.mean(map_ens))
	print("map mask rcnn",np.mean(map_rcnn))
	print("map unet",np.mean(map_unet))
	print("map ens",np.mean(map_ens))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.grid()
plt.plot(iou_t,xrcnn_iou,iou_t,xunet_iou,':',iou_t,xens_iou,'--')

In [ ]:
def dice_score(mask1, mask2):
    intersection = np.sum((mask1 + mask2)==2)
    summation=np.sum(mask1 + mask2)
    if summation==0:
        return 1
    score=(2*intersection)/summation
    return score
    
def map_t(gt_mask,pred_mask,iou_threshold=0.5):
    gt_mask=bin_mask_splitter(gt_mask)
    pred_mask=bin_mask_splitter(pred_mask)
    
    #if np.all(gt_mask)==0.0:
    #    if np.all(pred_mask)==0.0:
    #        return 1
    #    else:
    #        return 0
    ch_1 = gt_mask.shape[2]
    ch_2 = pred_mask.shape[2]
    
    p_match=-1*np.ones(ch_2)
    
    for i in range(ch_2):
        for j in range(ch_1):
            if np.any(gt_mask[:,:,j]+pred_mask[:,:,i]==2):
                IoU_score=iou_score(gt_mask[:,:,j], pred_mask[:,:,i])
            
                if IoU_score>=iou_threshold:
                    p_match[i]=1
                    
    precisions = np.cumsum(p_match > -1) / (np.arange(len(p_match)) + 1)
    recalls = np.cumsum(p_match > -1).astype(np.float32) / gt_mask.shape[2]
    
    precisions = np.concatenate([[0], precisions, [0]])
    recalls = np.concatenate([[0], recalls, [1]])

    # Ensure precision values decrease but don't increase. This way, the
    # precision value at each recall threshold is the maximum it can be
    # for all following recall thresholds, as specified by the VOC paper.
    for i in range(len(precisions) - 2, -1, -1):
        precisions[i] = np.maximum(precisions[i], precisions[i + 1])

    # Compute mean AP over recall range
    indices = np.where(recalls[:-1] != recalls[1:])[0] + 1
    mAP_val = np.sum((recalls[indices] - recalls[indices - 1]) *
                 precisions[indices])
    
    return mAP_val
    
def precision_recall(gt_mask, pred_mask, iou_threshold=0.5):    
    gt_mask=bin_mask_splitter(gt_mask)
    pred_mask=bin_mask_splitter(pred_mask)
    
    ch_1 = gt_mask.shape[2]
    ch_2 = pred_mask.shape[2]
    
    tp=0
    
    for i in range(ch_2):
        for j in range(ch_1):
            if np.any(gt_mask[:,:,j]+pred_mask[:,:,i]==2):
                IoU_score=iou_score(gt_mask[:,:,j], pred_mask[:,:,i])
            
                if IoU_score>=iou_threshold:
                    tp=tp+1
    fp=ch_2-tp                
    recall=tp/ch_1
    prec=tp/(tp+fp)
    return (prec,recall)
    
mask_unet1 = np.array([[0,0,0,0,0,0],[0,1,1,0,0,0],[0,0,1,1,0,0],[0,0,0,0,0,0]])
mask_unet2 = np.array([[0,0,0,0,0,1],[0,0,1,0,1,1],[0,0,1,1,0,1],[0,0,0,0,0,0]])

dice_score(mask_unet1,mask_unet2)

In [ ]:
grid_search()

In [ ]:
import skopt
from skopt import gp_minimize


def f(w):
    x=w[0]-w[1]
    return x


res = gp_minimize(ensemble_prediction,                  # the function to minimize
                  [(0.0, 1.0),(0.0, 1.0)],      # the bounds on each dimension of x
                  acq_func="EI",      # the acquisition function
                  n_calls=20,         # the number of evaluations of f
                  n_random_starts=5,  # the number of random initialization points
                  random_state=1234)

res.x[0],res.x[1], res.fun


In [ ]:
config.DETECTION_MIN_CONFIDENCE